In [7]:
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
from kabsch import kabsch_numpy

def generate_methanol(seed: int):
    # 1. 根据SMILES创建分子
    smiles = 'CO'  # 甲醇的SMILES
    mol = Chem.MolFromSmiles(smiles)
    # 2. 添加H原子
    mol = Chem.AddHs(mol)
    # 3. 生成三维坐标
    mol1 = AllChem.EmbedMolecule(mol, randomSeed=seed)
    # 4. 优化几何结构
    AllChem.UFFOptimizeMolecule(mol)
    
    return mol

def get_coords(mol: Chem.Mol):
    coords = []
    conf = mol.GetConformer()
    print(f"Number of atoms: {mol.GetNumAtoms()}")
    # 打印坐标信息
    for i, atom in enumerate(mol.GetAtoms()):
        pos = conf.GetAtomPosition(i)
        print(f"Atom {atom.GetSymbol()} {i}: {pos.x:.2f}, {pos.y:.2f}, {pos.z:.2f}")
        coords.append(np.array([pos.x, pos.y, pos.z]))

    return np.array(coords)


In [8]:
mol1 = generate_methanol(42)
P = get_coords(mol1)
mol2 = generate_methanol(47)
Q = get_coords(mol2)


Number of atoms: 6
Atom C 0: -0.36, 0.01, -0.02
Atom O 1: 0.91, -0.53, -0.26
Atom H 2: -0.55, 0.07, 1.07
Atom H 3: -0.43, 1.02, -0.48
Atom H 4: -1.13, -0.65, -0.48
Atom H 5: 1.56, 0.08, 0.17
Number of atoms: 6
Atom C 0: -0.36, 0.00, -0.02
Atom O 1: 0.92, -0.53, -0.24
Atom H 2: -0.55, 0.11, 1.07
Atom H 3: -0.44, 0.99, -0.52
Atom H 4: -1.12, -0.68, -0.46
Atom H 5: 1.55, 0.11, 0.17


In [9]:
def random_rotation_matrix(dim=3):
    # 随机生成一个方阵
    random_matrix = np.random.randn(dim, dim)
    # 使用QR分解生成一个正交矩阵
    q, r = np.linalg.qr(random_matrix)
    # 为确保行列式为 1，调整符号
    d = np.linalg.det(q)
    q = q * np.sign(d)
    
    return q

R_random = random_rotation_matrix()
t = np.random.randn(3)
Q = Q @ R_random + t

print('P:', P)
print('Q:', Q)


P: [[-0.35770023  0.00759022 -0.02148174]
 [ 0.90873557 -0.53499245 -0.26111898]
 [-0.54683347  0.07179144  1.07210873]
 [-0.43376811  1.01934375 -0.47579473]
 [-1.12699742 -0.64793055 -0.47895646]
 [ 1.55656366  0.08419759  0.16524319]]
Q: [[ 1.21105579  0.25292646 -0.59910266]
 [ 0.3647015   0.21101923 -1.71158191]
 [ 0.6190353   0.41775761  0.32727401]
 [ 1.78226174 -0.69712487 -0.51568857]
 [ 1.92790993  1.09117657 -0.7185453 ]
 [-0.2527862  -0.55029977 -1.55677488]]


In [10]:
def rmsd(P, Q):
    return np.sqrt(np.mean(np.sum((P - Q) ** 2, axis=1)))
print(rmsd(P, Q))


2.406919937051122


In [11]:
R_, t_, rmsd = kabsch_numpy(P, Q)
print(rmsd)

1.8810497549980157e-06
